In [1]:
import polars as pl
import psycopg
import config
import json

In [2]:
def build_uri(user, password, db, server = '127.0.0.1', port = 5433):
    return f"postgresql://{user}:{password}@{server}:{port}/{db}"

def build_query(table: str):
    return f"""SELECT * FROM {table}"""
    

In [3]:
query_sport = f"""
    SELECT 
        id_salarie,
        date_debut,
        date_fin,
        sport_type,
        distance_metre
    FROM 
        {config.SPORT_TABLE}"""


query_rh = f"""
    SELECT
        id_salarie,
        nom,
        prenom,
        salaire_brut,
        moyen_de_deplacement,
        pratique_d_un_sport,
        work_distance
    FROM
        {config.RH_TABLE}
    """

In [4]:
uri = build_uri(
    user = config.POSTGRES_USER, 
    password = config.POSTGRES_PASSWORD, 
    db = config.POSTGRES_DB,
    port = config.POSTGRES_PORT
    )

with psycopg.connect(uri) as pg_conn:
    df_sport = pl.read_database(query_sport, pg_conn)
    df_rh = pl.read_database(query_rh, pg_conn)


In [5]:
df_rh.glimpse()

Rows: 161
Columns: 7
$ id_salarie           <i64> 59019, 19841, 56482, 21886, 81001, 17757, 17036, 36913, 79006, 62296
$ nom                  <str> 'Colin', 'Ledoux', 'Dumont', 'Toussaint', 'Bailly', 'Bazin', 'Jacques', 'Pons', 'Rousset', 'Chauvin'
$ prenom               <str> 'Audrey', 'Monique', 'Michelle', 'Judith', 'Michelle', 'Margaret', 'Julien', 'Brigitte', 'Jérome', 'Zakaria'
$ salaire_brut         <i64> 30940, 74360, 51390, 70320, 46870, 31460, 72480, 59790, 29240, 68680
$ moyen_de_deplacement <str> 'Transports en commun', 'véhicule thermique/électrique', 'véhicule thermique/électrique', 'Marche/running', 'Marche/running', 'Vélo/Trottinette/Autres', 'véhicule thermique/électrique', 'Transports en commun', 'véhicule thermique/électrique', 'Vélo/Trottinette/Autres'
$ pratique_d_un_sport  <str> None, None, 'Tennis', None, None, 'Badminton', 'Escalade', None, None, None
$ work_distance        <i64> 26707, 22640, 48717, 1628, 803, 5173, 38546, 21728, 21924, 9116



In [ ]:
df_rh = df_rh.with_columns(
    pl.when(
        ((pl.col("work_distance") > 15000) & (pl.col("moyen_de_deplacement") == "Marche/running"))
        | ((pl.col("work_distance") > 25000) & (pl.col("moyen_de_deplacement") == "Vélo/Trottinette/Autres"))
    )
    .then(pl.lit("wrong_transport"))
    .otherwise(pl.col("moyen_de_deplacement"))
    .alias("moyen_de_deplacement")
)

In [16]:
df_rh.filter(pl.col("moyen_de_deplacement") == 'wrong_transport')

id_salarie,nom,prenom,moyen_de_deplacement,pratique_d_un_sport,work_distance,prime_sportive,montant_prime,nb_activites_2025,prime_cp,nb_prime_cp
str,str,str,str,str,i64,bool,f64,u32,bool,i32


In [6]:
df_rh = df_rh.with_columns(
    pl.col("moyen_de_deplacement")
    .is_in(["Marche/running", "Vélo/Trottinette/Autres"])
    .alias("prime_sportive")
)

In [7]:
df_rh = df_rh.with_columns([
    pl.when(pl.col("moyen_de_deplacement").is_in(["Marche/running", "Vélo/Trottinette/Autres"]))
      .then(pl.col("salaire_brut") * config.TAUX_PRIME)        
      .otherwise(0)
      .alias("montant_prime")
])

In [8]:
df_rh = df_rh.drop("salaire_brut")

In [9]:
df_2025 = df_sport.filter(pl.col("date_debut").dt.year() == 2025)

df_count_2025 = (
    df_2025
    .group_by("id_salarie")
    .agg(pl.len().alias("nb_activites_2025"))
)


In [10]:
df_rh = df_rh.with_columns(pl.col("id_salarie").cast(pl.Utf8))
df_count_2025 = df_count_2025.with_columns(pl.col("id_salarie").cast(pl.Utf8))

df_rh = df_rh.join(
    df_count_2025,
    on="id_salarie",
    how="left"
).with_columns(
    pl.when(pl.col("nb_activites_2025") > 15)
      .then(True)
      .otherwise(False)
      .alias("prime_cp")
).with_columns(
    pl.when(pl.col("prime_cp") == True)
      .then(config.NB_CP_PRIME)
      .otherwise(0)
      .alias("nb_prime_cp")
)

In [11]:
df_rh.null_count()

id_salarie,nom,prenom,moyen_de_deplacement,pratique_d_un_sport,work_distance,prime_sportive,montant_prime,nb_activites_2025,prime_cp,nb_prime_cp
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,66,0,0,0,66,0,0


In [12]:
df_sport.glimpse()

Rows: 9818
Columns: 5
$ id_salarie              <str> '56482', '56482', '56482', '56482', '56482', '56482', '56482', '56482', '56482', '56482'
$ date_debut     <datetime[μs]> 2024-06-20 08:08:00, 2024-06-19 08:11:00, 2024-06-24 17:36:00, 2024-06-23 14:51:00, 2024-06-24 18:39:00, 2024-07-08 07:24:00, 2024-07-17 19:10:00, 2024-07-22 14:10:00, 2024-08-05 10:20:00, 2024-08-12 06:06:00
$ date_fin       <datetime[μs]> 2024-06-20 09:36:00, 2024-06-19 09:17:00, 2024-06-24 18:55:00, 2024-06-23 16:17:00, 2024-06-24 19:28:00, 2024-07-08 09:17:00, 2024-07-17 20:53:00, 2024-07-22 15:01:00, 2024-08-05 12:05:00, 2024-08-12 07:47:00
$ sport_type              <str> 'Tennis', 'Tennis', 'Tennis', 'Tennis', 'Tennis', 'Tennis', 'Tennis', 'Tennis', 'Tennis', 'Tennis'
$ distance_metre          <i64> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0



In [13]:
df_rh.write_database(
    table_name=config.PBI_RH_TABLE,
    connection=uri,
    if_table_exists='replace'
)

df_sport.write_database(
    table_name=config.PBI_SPORT_TABLE,
    connection=uri,
    if_table_exists='replace'
)

818